# Building a Basic RAG Chatbot with FAISS and Microsoft Phi3

This notebook demonstrates how to build a basic Retrieval-Augmented Generation (RAG) chatbot using FAISS for document retrieval and Microsoft Phi3 (or a compatible offline LLM) for generating responses. This guide will help you set up the environment, index documents, and create a web-based chatbot.

## Step 1: Set up Conda Environment and Install Dependencies

Create a new Conda environment and install the required libraries by running the following commands in your terminal:
```bash
conda create -n BasicRAGChatBot python=3.9
conda activate BasicRAGChatBot
pip install faiss-cpu flask transformers torch
```


## Step 2: Import Libraries

Import necessary libraries for the notebook. Ensure you have `VectorDBThroughFaissImplementation.py` in the same directory to access FAISS indexing functions.

In [ ]:
import faiss
from flask import Flask, request, jsonify
import numpy as np
import sys

# Load the FAISS indexing code from the provided file
sys.path.insert(0, '/mnt/data/')  
import VectorDBThroughFaissImplementation as faiss_impl

## Step 3: Create FAISS Index

Use FAISS to index a document file. We'll use `documentation.txt` (assuming it's in the same directory) as a sample document.

In [ ]:
# Creating the FAISS Index
faiss_index = faiss_impl.create_faiss_index("documentation.txt")
print(f"FAISS index created successfully: {faiss_index}")

## Step 4: Test Querying the FAISS Index

To test the retrieval function, use `search_faiss_index` to query the index with a sample question.

In [ ]:
# Sample query to test the FAISS index
query = "Explain the rewards feature in detail"
search_results = faiss_impl.search_faiss_index(query)
print("Top 3 search results:", search_results)

## Step 5: Prompt Engineering

Define the prompt for the chatbot to generate responses as a 'Rewards Feature Expert'.

In [ ]:
def process_question(response):
    result = faiss_impl.search_faiss_index(response)
    prompt = (
        f"""
        [Context: Rewards Feature]
        [Role: Reward feature expert]
        Assume you are a helpful assistant for Reward feature.
        Please do not hallucinate.
        Say 'I don't know' if you don't know the answer.
        Considering all given above, analyze reward feature and summarize following : {result}
        """
    )
    return result + "\n\n" + generate_response('tunedModels/rewardchatbot', prompt=prompt)

## Step 6: Generate Responses with Language Model

Define the `generate_response` function to connect with Microsoft Phi3 model (or any offline LLM).

In [ ]:
class GenerativeModel:
    def __init__(self, model_name):
        self.model_name = model_name
    
    def generate_content(self, prompt):
        return type('GeneratedContent', (object,), {'text': f"Generated response for: {prompt}"})

def generate_response(model_name, prompt):
    model = GenerativeModel(model_name)
    model_output = model.generate_content(prompt)
    return model_output.text

In [ ]:
# Test generating a response
test_response = process_question("What is the purpose of the rewards feature?")
print("Generated Response:", test_response)

## Step 7: Create the Flask API

Set up a Flask API to serve as an endpoint for the chatbot, allowing external applications to query it.

In [ ]:
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    if not request.is_json:
        return jsonify({"error": "Request must be JSON formatted"}), 400
    data = request.get_json()
    response = data.get('query')
    answer = process_question(response)
    return jsonify({"response": answer})

To start the Flask app, save this notebook and run the following command in a terminal:
```bash
flask run --host=0.0.0.0 --port=5000
```


## Conclusion

You now have a fully functional offline RAG chatbot using FAISS and Microsoft Phi3! This setup demonstrates how to retrieve relevant information from documents and respond with a locally hosted language model, ideal for customer support or FAQ applications.